In [76]:
import numpy as np

In [ ]:
class Tensor():
    def __init__(self, data, children=(), op=''):
        self.data: np.ndarray = np.array(data)
        self.grad = np.zeros_like(data)
        self._prev = set(children)
        self._backward = lambda : None
        self._op = op

    @property
    def shape(self):
        return self.data.shape
    
    @property
    def size(self): 
        return self.data.size

    # need to build topo graph and then go through it and call backwards on each of the tensors
    def backward(self):
        self.grad = np.ones_like(self.data)
        
        topo = []
        visited = set()

        # do DFS on un-visited nodes, add node to topo-when all its children have been visited
        def build_topo(node):
            if node not in visited:
                visited.add(node)
                for child in node._prev:
                    build_topo(child)
                topo.append(node)
        build_topo(self)

        for node in reversed(topo):
            node._backward()

            
    def __add__(self, operand):
        operand = operand if isinstance(operand, Tensor) else Tensor(operand)
        out = Tensor(self.data + operand.data, (self, operand), '+')

        def _backward():
            self.grad += out.grad
            operand.grad += out.grad

        out._backward = _backward

        return out

    def __neg__(self):
        out = Tensor(-self.data, (self,), '-')

        def _backward():
            self.grad = -out.grad
        out._backward = _backward
        return out
    
    def __sub__(self, operand):
        return self + (-operand)

    # only support tensor ** scalar for now, not supporting reversed
    def __mul__(self, operand):
        if not isinstance(operand, float):
            raise NotImplementedError(f'* not implemented between tensor and {type(operand)}')
        out = Tensor(self.data*operand, (self,), f'{operand}*')

        def _backward():
            self.grad = out.grad * operand
        out._backward = _backward
        return out
        
    # only support tensor ** scalar for now, not supporting reversed
    def __truediv__(self, operand):
        if not isinstance(operand, float):
            raise NotImplementedError(f'/ not implemented between tensor and {type(operand)}')
        return self * (operand**-1)
    
    # only support tensor ** scalar for now, not supporting reversed
    def __pow__(self, operand):
        if not isinstance(operand, float):
            raise NotImplementedError(f'** not implemented between tensor and {type(operand)}')
        out = Tensor(self.data*operand, (self,), f'**{operand}')

        def _backward():
            self.grad = out.grad * ((operand)*(self.data**(operand-1)))
        out._backward = _backward
        return out
    
    def __matmul__(self, operand):
        operand = operand if isinstance(operand, Tensor) else Tensor(operand)
        out = Tensor(self.data @ operand.data, (self, operand), '@')

        def _backward():
            self.grad += out.grad @ operand.data.T
            operand.grad += self.data.T @ out.grad
        out._backward = _backward
        return out
    
    def relu(self):
        out = Tensor((self.data > 0) * self.data, (self,), 'Relu')

        def _backward():
            self.grad = (self.data > 0) * out.grad
        out._backward = _backward
        return out
    
    def sum(self):
        out = Tensor(sum(self.data), (self,), 'sum')

        def _backward():
            self.grad = np.full_like(self.data, out.grad)
        out._backward = _backward
        return out

    def mean(self):
        return (self.sum)/float(self.size)
    
    def __repr__(self):
        return f'tensor: {self.data}, grad: {self.grad}, op:{self._op}'

class Relu():
    def __call__(self, input):
        if isinstance(input, Tensor):
            return input.relu()
        else:
            raise TypeError('Input type: {input.type}, not supported')
        


In [ ]:
a = Tensor(np.arange(4).reshape((2,2)))
b = Tensor(np.arange(4).reshape((2,2)))
c = Tensor(np.arange(4).reshape((2,2)))

d = a @ b
e = d + c


In [101]:
e.backward()
a

tensor: [[0 1]
 [2 3]], grad: [[ 91 455]
 [ 91 455]], op:

In [80]:
a = Tensor(np.array([-1,2,3,4,-12]))
a.data
b = a.sum()
b.data

array(-4)

In [81]:
RNG = np.random.default_rng()

'''generate random dimenstions for input size,
test given operator, generate random input, run 
forward through it and a slightly shitfed version,
ensure that the backward pass gradient matches the 
finite differences gradient buy only a small amount.'''
def test_tensor_op(op='+'):
    a, b, c = RNG.integers(size=(3))
    input1 = RNG.random(size=((a,b)))
    input2 = RNG.random(size=(b,c))

    perturbed_1

In [82]:
dir(np.ndarray)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_namespace__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__buffer__',
 '__class__',
 '__class_getitem__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
